### Stable Diffusion Generations for Survey

In [1]:
#pip install stability-sdk

In [1]:
import os
import io
import warnings
from PIL import Image
from stability_sdk import client
import stability_sdk.interfaces.gooseai.generation.generation_pb2 as generation

#### Part 1 - Access Stability AI Account

In [2]:
# Our Host URL should not be prepended with "https" nor should it have a trailing slash.
os.environ['STABILITY_HOST'] = 'grpc.stability.ai:443'

In [3]:
os.environ['STABILITY_KEY'] = 'sk-xOUcYTrHZcjxjrssW9ajggI1tjNEyt1yCqbaSPVhyEcvazLq'

In [4]:
# Set up our connection to the API.
stability_api = client.StabilityInference(
    key=os.environ['STABILITY_KEY'], # API Key reference.
    verbose=True, # Print debug messages.
    engine="stable-diffusion-v1-5", # Set the engine to use for generation. 
    # Available engines: stable-diffusion-v1 stable-diffusion-v1-5 stable-diffusion-512-v2-0 stable-diffusion-768-v2-0 
    # stable-diffusion-512-v2-1 stable-diffusion-768-v2-1 stable-inpainting-v1-0 stable-inpainting-512-v2-0
)

#### Part 2 - Functions for Image Processing

In [5]:
def choosePrompt(sbj):
    medium = ["painting", "photograph", "3D render"] #3
    subject = ["person", "forest", "dog"] #3
    style = ["Baroque", "Neo-Plasticism", "digital art"] #3

    #except for prompt with verb - do it seperate
    structure_1 = "<MDM> of <SBJ> in the style of <SYL>"
    structure_2 = "<SYL> <MDM> of <SBJ>"
    structure_3 = "<MDM> of <SBJ>, <SYL>"
    structure_4 = "<MDM> of <SBJ> with <SYL> style"
    structure_5 = "<MDM>, <SBJ>, <SYL>"
    structure_6 = "<SBJ>, <MDM>, <SYL>"

    structures = [structure_1, structure_2,structure_3,structure_4,structure_5,structure_6]

    # medium - subject - style
    countRep = len(medium) + len(style)
    sbj_prompt_list = []
    tmp_prompt = ""
    title = []
    #painting biss
    for i in range(len(style)): #3
        for j in range(len(medium)): #3
            for k in range(len(structures)): #3
                title.append(sbj + "_" + style[i] + "_" + medium[j] + "_" + "struct" + str(k)+".png")
                tmp_prompt = structures[k].replace("<MDM>", medium[j])
                tmp_prompt = tmp_prompt.replace("<SYL>", style[i])
                tmp_prompt = tmp_prompt.replace("<SBJ>", sbj)
                sbj_prompt_list.append(tmp_prompt)  

    return sbj_prompt_list, title

In [6]:
#Set Prompts
sbj1 = "person"
sbj2 = "forest"
sbj3 = "dog"

person_list, person_title = choosePrompt(sbj1)
forest_list, forest_title = choosePrompt(sbj2)
dog_list, dog_title = choosePrompt(sbj3)

#### Part 3 - Set Model Parameters

In [7]:
# Set up our initial generation parameters.
def setParameters(prompt):
    answers = stability_api.generate(
        prompt=prompt,
        seed=0, # If a seed is provided, the resulting generated image will be deterministic.
                        # What this means is that as long as all generation parameters remain the same, you can always recall the same image simply by generating it again.
                        # Note: This isn't quite the case for Clip Guided generations, which we'll tackle in a future example notebook.
        steps=30, # Amount of inference steps performed on image generation. Defaults to 30. 
        cfg_scale=7.0, # Influences how strongly your generation is guided to match your prompt.
                       # Setting this value higher increases the strength in which it tries to match your prompt.
                       # Defaults to 7.0 if not specified.
        width=512, # Generation width, defaults to 512 if not included.
        height=512, # Generation height, defaults to 512 if not included.
        samples=1, # Number of images to generate, defaults to 1 if not included.
        sampler=generation.SAMPLER_K_DPMPP_2M # Choose which sampler we want to denoise our generation with.
                                                     # Defaults to k_dpmpp_2m if not specified. Clip Guidance only supports ancestral samplers.
                                                     # (Available Samplers: ddim, plms, k_euler, k_euler_ancestral, k_heun, k_dpm_2, k_dpm_2_ancestral, k_dpmpp_2s_ancestral, k_lms, k_dpmpp_2m)
    )
    return answers

#### Part 4 - Generating the images

In [44]:
# Set up our warning to print to the console if the adult content classifier is tripped.
# If adult content classifier is not tripped, save generated images.
def genImage(sbj_list, sbj_title):
    for i in range(len(sbj_list)):
        answers = setParameters(sbj_list[i])
        for resp in answers:
            for artifact in resp.artifacts:
                if artifact.finish_reason == generation.FILTER:
                    warnings.warn(
                        "Your request activated the API's safety filters and could not be processed."
                        "Please modify the prompt and try again.")
                if artifact.type == generation.ARTIFACT_IMAGE:
                    img = Image.open(io.BytesIO(artifact.binary))
                    img.save(str(sbj_title[i])+ ".png") # Save our generated images with their seed number as the filename.

In [43]:
genImage(person_list, person_title)
genImage(forest_list, forest_title)
genImage(dog_list, dog_title)